# Digital Twin Simulation Demo

This notebook demonstrates the digital twin simulator for CKD progression forecasting.


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.preprocess.features import load_data, build_sequence_data
from src.models.lstm_model import LSTMEGFR
import torch

# Load data and model
meta, labs, meds, events = load_data("data/synthetic")
X_seq, X_tab, pids = build_sequence_data(meta, labs, seq_len=90)

# Load trained model (if available)
model_path = "outputs/lstm_model.pt"
device = "cuda" if torch.cuda.is_available() else "cpu"

if os.path.exists(model_path):
    model = LSTMEGFR(seq_input_dim=1, tab_input_dim=5)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    print("Model loaded successfully")
else:
    print("Model not found. Please train the model first.")


In [ ]:
# Example: Forecast for a patient
from src.simulator.twin import forecast_patient, load_model

if os.path.exists(model_path):
    model = load_model(model_path, device)
    patient_id = pids[0]
    preds = forecast_patient(patient_id, model, X_seq, X_tab, list(pids), horizon=90, device=device)
    
    # Plot forecast
    plt.figure(figsize=(10, 6))
    plt.plot(preds, label='Forecasted eGFR')
    plt.xlabel('Days')
    plt.ylabel('eGFR (mL/min/1.73m²)')
    plt.title(f'CKD Progression Forecast for {patient_id}')
    plt.legend()
    plt.grid(True)
    plt.show()
